In [ ]:
import numpy as np
import cv2, math, time
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

# Only for jupyter notebook visualization
%matplotlib inline 

In [ ]:
img = cv2.imread("Images\saw_01.png", 0)

# Histogram of the image
plt.yticks([])
hist,bin = np.histogram(img.ravel(),256,[0,255])
plt.xlim([0,255])
plt.plot(hist)
plt.title('histogram')
plt.show()

# Apply binary thresholding
ret,img_thresholded = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
plt.imshow(img_thresholded, cmap = 'gray')
plt.title('Binary thresholding')
plt.xticks([]), plt.yticks([])
plt.show()

# # Otsu's thresholding after Gaussian filtering
# blur = cv2.GaussianBlur(img,(5,5),0)
# ret,img_thresholded = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# plt.imshow(img_thresholded, cmap = 'gray')
# plt.title('Otsu\'s thresholding')
# plt.xticks([]), plt.yticks([])
# plt.show()

# Do erosion in order to estract contours
kernel = np.ones((3,3),np.uint8)
img_contours = img_thresholded - cv2.erode(img_thresholded, kernel)     # Contours are white, background is black
img_contours_negative = 255 - img_contours                              # Contours are black, background is white
plt.subplot(1,2,1)
plt.imshow(img_contours, cmap = 'gray')
plt.title('Contours')
plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2)
plt.imshow(img_contours_negative, cmap = 'gray')
plt.title('Contours Negative')
plt.xticks([]), plt.yticks([])
plt.show()

# Show contours in the original image as a WHITE overlay
img_final = cv2.bitwise_or(img, img_contours)
plt.imshow(img_final, cmap = 'gray')
plt.title('Final image')
plt.xticks([]), plt.yticks([])
plt.show()

#cv2.imwrite("Images\saw_01_app_contours.png", img_final)


In [ ]:
# Implementation of RDP Algorithm (Optimized)
def RDP_Algorithm(points, epsilon):
    # get the start and end points
    start = points[0]
    end = points[-1]

    # find distance from other points to line formed by start and end
    dist_point_to_line = DPTL(points, start, end)

    # get the index of the points with the largest distance
    max_value = max(dist_point_to_line)
    max_idx = dist_point_to_line.index(max_value) + 1 #since the first (and the last point) are not included in the calculation

    result = []
    if max_value > epsilon:
        if len(points[:max_idx+1]) == 2:
            result += [list(i) for i in points[:max_idx+1] if list(i) not in result]
        else:
            partial_results_left = RDP_Algorithm(points[:max_idx+1], epsilon)
            result += [list(i) for i in partial_results_left if list(i) not in result]
        if len(points[max_idx:]) == 2:
            result += [list(i) for i in points[max_idx:] if list(i) not in result]
        else:
            partial_results_right = RDP_Algorithm(points[max_idx:], epsilon)
            result += [list(i) for i in partial_results_right if list(i) not in result]
    else:
        result += [points[0], points[-1]]
    
    return result

def DPTL(points, start, end):
    # return a list of distances: distance of each point in points to line formed by start and end

    # compute the angular coefficient and the constant of the line formed by start and end
    # y - mx - q = 0
    a = start[1] - end[1]
    b = end[0] - start[0]
    c = - a*start[0] - b*start[1] 

    return [abs(a*points[i][0]+b*points[i][1]+c)/(math.sqrt(a**2+b**2)) for i in range(1,len(points)-1)]

If the contour points do not follow a consistent left-to-right order, you may need to implement a custom sorting method that respects the connectivity of the contour. One approach is to start from a known point and iteratively find the next point in the contour based on its connectivity to the previous point. 

In [ ]:
# Implementation of the custom sorting algorithm
def edge_chain_sort(points, img, contour_color = 255):
    # sort:  convert an edge map into chains of adjacent (continuous) edge pixels
    # using 8 connectivity
    dx = [-1, 0, 1, 1, 1, 0, -1, -1]
    dy = [-1, -1, -1, 0, 1, 1, 1, 0]

    points = sorted(points, key=lambda x: x[1])

    current_pixel = points[0]
    chain = [current_pixel]
    previous_pixel = [-1,-1]

    while (previous_pixel != current_pixel):
        previous_pixel = current_pixel

        for j in range(8):
            # Calculate the neighbor's coordinates
            neighbor_x = current_pixel[1] + dx[j]
            neighbor_y = current_pixel[0] + dy[j]
            
            # Check if the neighbor is within the image bounds and is an edge pixel and not already in "chain"
            if (0 <= neighbor_x < img.shape[1] and 0 <= neighbor_y < img.shape[0]):
                if (img[neighbor_y, neighbor_x] == contour_color and [neighbor_y, neighbor_x] not in chain):
                    current_pixel = [neighbor_y, neighbor_x]
                    chain.append(current_pixel)
                    break
    return chain

Sort a list of contour points based on their connectivity.

Args:
- points: List of tuples representing the contour points [(x1, y1), (x2, y2), ...]

Returns:
- Sorted list of contour points

In [ ]:
def sort_contour_points(points):
    sorted_points = sorted(points, key=lambda p: p[1])
    start_point = sorted_points[0]
    sorted_points.remove(start_point)
    sorted_contour = [start_point]

    while sorted_points:
        last_point = sorted_contour[-1]

        euclidean_distances = [((last_point[0] - p[0]) ** 2 + (last_point[1] - p[1]) ** 2) ** 0.5 for p in sorted_points]
        min_distance = min(euclidean_distances)
        nearest_neighbors = [i for i in range(len(euclidean_distances)) if euclidean_distances[i] == min_distance]

        for i in sorted(nearest_neighbors, reverse=True):
            sorted_contour.append(sorted_points[i])
            sorted_points.remove(sorted_points[i])
            
    return sorted_contour

'''
Why has been necessary to sort the points of the contour?
    - The contour is a list of points that are not ordered in any way, so the first step is to order them in a way that
    makes sense. The order of the points is important because it will be used to calculate the angle between each pair of
    points, and the angle is the most important feature of the contour.
Why has been necessary to use the euclidean distance to sort the points?
    - The euclidean distance is used to find the nearest neighbor of a point, and it is used because it is the most
    straightforward way to find the nearest neighbor of a point in a 2D space.
Why has been necessary to use the nearest neighbors?
    - Because the nearest neighbors of a point are the next points in the contour, and it is important to find the nearest
    neighbors of a point to order the contour. 
'''

sorted_contour = sort_contour_points(raw_contour)

In [ ]:
# applying RDP algorithm to the image contours
img_cc = img_contours_negative[350:600,550:]
plt.imshow(img_cc, cmap = 'gray')
plt.title('Contours Cropped')
plt.xticks([]), plt.yticks([])
plt.show()

start = time.time()

contours = [[i,j] for i in range(img_cc.shape[0]) for j in range(img_cc.shape[1]) if img_cc[i,j] == 0]

contours = sort_contour_points(contours)

simplified_contours = RDP_Algorithm(contours, epsilon = 10)

simplified_contours = np.array(simplified_contours)

plt.plot(simplified_contours[:,1], simplified_contours[:,0], 'b')
plt.imshow(img_cc, cmap = 'gray')
plt.title('Simplified contours')
plt.xticks([]), plt.yticks([])
plt.show()

cv2.imwrite("Images\saw_01_simplified_contours.png", img_cc)